In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
!python report_json.py

MiET
1 
N2OB
1 
Legacy
1 
Finish!


In [3]:
with open('./jsonFiles/allconfig.json', encoding='utf-8') as f:
    allconfig = json.load(f)

### initialization part ###
Controllist = allconfig['control']
ss= allconfig['scenario']
dropnum = allconfig['drop_num']
ssinfo = allconfig['scenario_info']

In [4]:
def dict_to_dframe(datadict):
    var_df = pd.DataFrame()
    for s,S in datadict.items():
        for control,CONTROL in S.items():
            for drop,DROP in CONTROL.items():
                for node,NODE in DROP.items():
                    keys = []
                    vals = []
                    for k,v in NODE.items():#print(k,v)
                        keys.append(k)
                        vals.append(v)
                    tmp_se = pd.Series(vals, index = keys)
                    var_df = var_df.append(tmp_se,ignore_index=True)
    return var_df

In [5]:
def list_to_dict(df):
    c_dict = {}
    for i in range(len(df.index[0]) - 1, -1 ,-1):
        p_dict = {}
        for index1 in range(0,len(list(set(df.index.get_level_values(i))))):
            p_dict[list(set(df.index.get_level_values(i)))[index1]] = c_dict.copy()
        c_dict = p_dict.copy()
    
    for s,S in c_dict.items():
        for act,ACT in S.items():
            for r,R in ACT.items():
                    if (s,act,r) in df.index:
                        c_dict[s][act][r] = df.loc[s,act,r].to_dict()
                    else:
                        del c_dict[s][act][r]
    return (c_dict)
        

In [6]:
datadict = {}
for control in Controllist:
    with open('./jsonFiles/extract_json_'+ ss + '_'+ control + '.json', encoding='utf-8') as f:
        datadict.update(json.load(f))

In [7]:
def concat_max_min(node,df):
    Max_Min = pd.DataFrame({'Node': [node,node],
                            'Max_Min': ['Max','Min']})
    
    df = df[df['act'] == node] if node in ['AP','STA'] else df
    df = pd.concat([df.max(), df.min()] ,axis=1).T
    df = pd.concat([df,Max_Min] , axis=1)
    return df

In [8]:
dframe = dict_to_dframe(datadict).replace(np.nan,0).drop(['x', 'y','z','Uflow_num','Dflow_num','drop','bss','control','inter','role'], axis=1)

In [9]:
dframe[dframe['Dsinr']==50.4]

,Dmcs,Dsinr,Dthr,Dtraf,Umcs,Usinr,Uthr,Utraf,act,assoc,...,notdetect,reover,rxtime,rxtimeme,rxtimeo,rxtimes,sent,ss,txp,txtime
289,6.0,50.4,0.0,0.0,0.0,6.1,0.0,0.0,STA,[51],...,0.0,0.0,3.3,0.7,2.2,0.4,2.0,ss5,0.0,0.0


In [10]:
dframe_AP = concat_max_min('AP',dframe.copy())
dframe_STA = concat_max_min('STA',dframe.copy())
dframe_COMMON = concat_max_min('Common',dframe.copy())

In [11]:
dframe_AP

,Dmcs,Dsinr,Dthr,Dtraf,Umcs,Usinr,Uthr,Utraf,act,assoc,...,rxtime,rxtimeme,rxtimeo,rxtimes,sent,ss,txp,txtime,Node,Max_Min
0,7,45.7,111.4,250,7,30.8,132.8,200,AP,"[448, 449, 450, 451]",...,12.5,12,3,0,190144,ss5,23,7.5,AP,Max
1,2.7,10.9,0.1,150,2,5.4,1.2,100,AP,"[92, 93, 94, 95]",...,1.4,0.6,0,0,136,ss5,-1.9,0.1,AP,Min


In [12]:
dframe_STA

,Dmcs,Dsinr,Dthr,Dtraf,Umcs,Usinr,Uthr,Utraf,act,assoc,...,rxtime,rxtimeme,rxtimeo,rxtimes,sent,ss,txp,txtime,Node,Max_Min
0,7,57.6,76.6,100,7,30.8,100,100,STA,[91],...,15.1,7,2.7,14.8,170715,ss5,15,12.1,STA,Max
1,0,6.5,0,0,0,5.4,0,0,STA,[2],...,0.6,0.2,0,0,2,ss5,-13.6,0,STA,Min


In [13]:
dframe_COMMON

,Dmcs,Dsinr,Dthr,Dtraf,Umcs,Usinr,Uthr,Utraf,act,assoc,...,rxtime,rxtimeme,rxtimeo,rxtimes,sent,ss,txp,txtime,Node,Max_Min
0,7,57.6,111.4,250,7,30.8,132.8,200,STA,"[448, 449, 450, 451]",...,15.1,12,3,14.8,190144,ss5,23,12.1,Common,Max
1,0,6.5,0,0,0,5.4,0,0,AP,[2],...,0.6,0.2,0,0,2,ss5,-13.6,0,Common,Min


In [14]:
pd.concat([dframe_AP, dframe_STA.drop(columns=['Dsinr', 'Dthr']), dframe_COMMON.drop(columns=['Dsinr', 'Dthr'])],axis = 0,sort=True)

,Dmcs,Dsinr,Dthr,Dtraf,Max_Min,Node,Umcs,Usinr,Uthr,Utraf,...,notdetect,reover,rxtime,rxtimeme,rxtimeo,rxtimes,sent,ss,txp,txtime
0,7,45.7,111.4,250,Max,AP,7,30.8,132.8,200,...,0,797.1,12.5,12,3,0,190144,ss5,23,7.5
1,2.7,10.9,0.1,150,Min,AP,2,5.4,1.2,100,...,0,20.6,1.4,0.6,0,0,136,ss5,-1.9,0.1
0,7,NaN,NaN,100,Max,STA,7,30.8,100,100,...,64,500.2,15.1,7,2.7,14.8,170715,ss5,15,12.1
1,0,NaN,NaN,0,Min,STA,0,5.4,0,0,...,0,0,0.6,0.2,0,0,2,ss5,-13.6,0
0,7,NaN,NaN,250,Max,Common,7,30.8,132.8,200,...,64,797.1,15.1,12,3,14.8,190144,ss5,23,12.1
1,0,NaN,NaN,0,Min,Common,0,5.4,0,0,...,0,0,0.6,0.2,0,0,2,ss5,-13.6,0


In [15]:
axisrange = pd.concat([dframe_AP, dframe_STA, dframe_COMMON],axis = 0,sort=True).set_index(['ss','Node','Max_Min'])

In [16]:
path = './jsonFiles/axisrange.json'
with open(path,encoding='utf-8') as ex:
    axisrange_dict = json.load(ex)

axisrange_dict.update(list_to_dict(axisrange))

In [17]:
with open(path, 'w') as ex:
    ex.write(json.dumps(axisrange_dict,indent=4))